![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Construcción e implementación de modelos basados en Boosting
En este notebook aprenderá a construir e implementar modelos de Adaboost, Gradient Boosting y XGBoost. El primer modelo lo desarrollará de forma manual y usando la librería especializada sklearn, mientras que los otros dos los desarrollará solamente usando la librería.

## Instrucciones Generales:

Los modelos que construirá por medio de este notebook deberán predecir si un usuario deja o no de usar los servicios de una compañía (churn) teniendo en cuenta diferentes variables. Para conocer más detalles de la base de 'churn' puede ingresar al siguiente vínculo: http://srepho.github.io/Churn/Churn

Para realizar la actividad, solo siga las indicaciones asociadas a cada celda del notebook. 

In [1]:
# Importar base de datos y librerías
import pandas as pd
import numpy as np

# Carga de datos de archivos .csv
data = pd.read_csv('https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/datasets/churn.csv')

In [2]:
# Selección de variables numéricas (X)
X = data.iloc[:, [1,2,6,7,8,9,10]].astype(np.float)
# Tranformación de variables booleanas a floats
X = X.join((data.iloc[:, [4,5]] == 'no').astype(np.float))

# Definición variable de interés binaria (y)
y = (data.iloc[:, -1] == 'True.').astype(np.int)

In [3]:
# Separación de variables predictoras (X) y variable de interés (y) en set de entrenamiento y test usandola función train_test_split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=40)
n_samples = X_train.shape[0]

## Adaboost manual

In [4]:
# Definición de la cantidad de árboles de decisión del modelo
n_estimators = 10

# Definición de DataFrame para guardar pesos de las observaciones en cada árbol de decisión
weights = pd.DataFrame(index=X_train.index, columns=list(range(n_estimators)))

# Asignación los mismos pesos para todas las observaciones en el árbol 0
t = 0
weights[t] = 1 / n_samples

# Visualización de DataFrame de pesos
weights.head()

,0,1,2,3,4,5,6,7,8,9
2953,0.000448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
617,0.000448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,0.000448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
853,0.000448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2510,0.000448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

# Definición y entrenamiento (fit) del primer árbol de decisión (DecisionTreeClassifier)
trees = []
trees.append(DecisionTreeClassifier(max_depth=1))
trees[t].fit(X_train, y_train, sample_weight=weights[t].values)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [6]:
# Estimación del error del primer árbol de decisión
y_pred_ = trees[t].predict(X_train)
error = []
error.append(1 - metrics.balanced_accuracy_score(y_pred_, y_train, weights[t].values))
error[t]

0.24114832535884934

In [7]:
# Cálculo del factor alpha del primer árbol de decisión
alpha = []
alpha.append(np.log((1 - error[t]) / error[t])/2)
alpha[t]

0.5731970670617188

In [8]:
# Actualización de los pesos a considerar en el segundo árbol (t+1)
weights[t + 1] = weights[t]
filter_ = y_pred_ != y_train

weights.loc[filter_, t + 1] = weights.loc[filter_, t] * np.exp(alpha[t])

In [9]:
# Normalización de los pesos
weights[t + 1] = weights[t + 1] / weights[t + 1].sum()

In [10]:
# Visualización de DataFrame de pesos
weights.head()

,0,1,2,3,4,5,6,7,8,9
2953,0.000448,0.000406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
617,0.000448,0.000406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,0.000448,0.000406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
853,0.000448,0.000406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2510,0.000448,0.000406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Definición de loop que itera sobre todos los árboles definidos realizando los mismos cáculos presentados anteriormente
for t in range(1, n_estimators):
    # Definición y entrenamiento (fit) del árbol t
    trees.append(DecisionTreeClassifier(max_depth=1))
    trees[t].fit(X_train, y_train, sample_weight=weights[t].values)
    y_pred_ = trees[t].predict(X_train)
    # Estimación del error del árbol t
    error.append(1 - metrics.balanced_accuracy_score(y_pred_, y_train, weights[t].values))
    # Cálculo del factor alpha para el árbol t
    alpha.append(np.log((1 - error[t]) / error[t]) / 2)
    # Actualización de pesos para el árbol t+2
    weights[t + 1] = weights[t]
    filter_ = y_pred_ != y_train
    weights.loc[filter_, t + 1] = weights.loc[filter_, t] * np.exp(alpha[t])
    weights[t + 1] = weights[t + 1] / weights[t + 1].sum()

C:\Users\Luisa.Roa\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1745: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
C:\Users\Luisa.Roa\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1745: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


In [12]:
# Visualización de los errores de cada árbol
error

[0.24114832535884934,
 0.31085674971292176,
 0.26303609328491306,
 0.3509920019261563,
 0.3778663388376744,
 0.3908839674420268,
 0.436104330420925,
 0.4291168064851212,
 0.229599990460706,
 0.38855949425356906]

In [13]:
# Visualización de DataFrame de pesos
weights.head()

,0,1,2,3,4,5,6,7,8,9,10
2953,0.000448,0.000406,0.000369,0.000313,0.000281,0.000254,0.000231,0.000248,0.000267,0.000194,0.000221
617,0.000448,0.000406,0.000369,0.000313,0.000281,0.000254,0.000231,0.000248,0.000267,0.000194,0.000221
26,0.000448,0.000406,0.000369,0.000313,0.000281,0.000254,0.000231,0.000218,0.000204,0.000148,0.000169
853,0.000448,0.000406,0.000369,0.000313,0.000281,0.000254,0.000231,0.000248,0.000267,0.000194,0.000221
2510,0.000448,0.000406,0.000369,0.000313,0.000281,0.000254,0.000231,0.000248,0.000267,0.000194,0.000221


In [14]:
# Solo se usan modelos con error menor a 0.5
new_n_estimators = np.sum([x<0.5 for x in error])

In [15]:
# Prección sobre la muestra de test
y_pred_all = np.zeros((X_test.shape[0], new_n_estimators))
for t in range(new_n_estimators):
    y_pred_all[:, t] = trees[t].predict(X_test)

In [16]:
# Obtención de la predicción final al poderar las predicciones por el factor aplha
y_pred = (np.sum(y_pred_all * alpha[:new_n_estimators], axis=1) >= 1).astype(np.int)

In [17]:
# Impresión del desempeño modelo
metrics.f1_score(y_pred, y_test.values), metrics.accuracy_score(y_pred, y_test.values)

(0.48184818481848185, 0.8572727272727273)

In [18]:
# Definición de un sólo árbol de decisón para compara el desempeño
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
metrics.f1_score(y_pred, y_test.values), metrics.accuracy_score(y_pred, y_test.values)

(0.3548387096774193, 0.8181818181818182)

## Adaboost usando sklearn

In [19]:
# Importación y definición de modelo AdaBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier()
clf

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=50, random_state=None)

In [20]:
# Entrenamiento (fit) y desempeño del modelo AdaBoostClassifier
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
metrics.f1_score(y_pred, y_test.values), metrics.accuracy_score(y_pred, y_test.values)

(0.36771300448430494, 0.8718181818181818)

## Gradient Boosting usando sklearn

In [21]:
# Importación y definición de modelo GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier()
clf

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [22]:
# Entrenamiento (fit) y desempeño del modelo GradientBoostingClassifier
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
metrics.f1_score(y_pred, y_test.values), metrics.accuracy_score(y_pred, y_test.values)

(0.4644549763033175, 0.8972727272727272)

## XGBoost usando sklearn
Instalar la librería xgboost usando el comando: pip install xgboost

In [23]:
# Importación y definición de modelo XGBClassifier

from xgboost import XGBClassifier
clf = XGBClassifier()
clf

XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='binary:logistic', random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None, subsample=None,
              tree_method=None, validate_parameters=None, verbosity=None)

In [24]:
# Entrenamiento (fit) y desempeño del modelo XGBClassifier
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
metrics.f1_score(y_pred, y_test.values), metrics.accuracy_score(y_pred, y_test.values)

(0.4298245614035088, 0.8818181818181818)